<a href="https://colab.research.google.com/github/mike-nut/NagaokaHackathon/blob/main/conclusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# アプリ

In [ ]:
!pip install gradio | tail -1
!pip install openai==0.28 | tail -1

In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
from datetime import datetime
import json
import openai
import urllib.parse

# OpenAIのURLを設定
openai.api_base = "https://api.openai.com/v1"

# APIキーを設定
openai.api_key = ""

## ユーザーデータ

In [ ]:
USERS_DB = {
    "users": [
        {
            "id": 1,
            "name": "酒好き太郎",
            "age": 28,
            "gender": "男性",
            "prefecture": "東京都",
            "favorite_sake": ["日本酒", "ビール"],  # チェックボックスで選択されるのでリスト
            "interests": ["食べ歩き", "酒蔵見学"],  # チェックボックスで選択されるのでリスト
            "character": "日本酒初心者ですが、色々な銘柄を試してみたいです！",
            "pass": "taro123" # パスワード
        },
        {
            "id": 2,
            "name": "日本酒花子",
            "age": 35,
            "gender": "女性",
            "prefecture": "新潟県",
            "favorite_sake": ["日本酒"],
            "interests": ["利き酒", "酒蔵巡り", "温泉"],
            "character": "地元新潟の日本酒を愛しています！",
            "pass": "hanako456" # パスワード
        },
        {
            "id": 3,
            "name": "吟醸次郎",
            "age": 42,
            "gender": "男性",
            "prefecture": "長野県",
            "favorite_sake": ["日本酒", "ワイン (白)"],
            "interests": ["旅行", "写真", "ワイン"],
            "character": "フルーティーな香りの吟醸酒が好きです。",
            "pass": "jiro789" # パスワード
        },
        {
            "id": 4,
            "name": "純米三郎",
            "age": 31,
            "gender": "男性",
            "prefecture": "京都府",
            "favorite_sake": ["日本酒"],
            "interests": ["料理", "歴史"],
            "character": "米の旨味がしっかりした純米酒が好き。",
            "pass": "saburo101" # パスワード
        },
        {
            "id": 5,
            "name": "本醸造美咲",
            "age": 25,
            "gender": "女性",
            "prefecture": "大阪府",
            "favorite_sake": ["焼酎", "梅酒"],
            "interests": ["音楽", "映画", "スイーツ"],
            "character": "普段飲みできるお手頃な日本酒が好きです。",
            "pass": "misaki222" # パスワード
        },
        {
            "id": 6,
            "name": "大吟醸一郎",
            "age": 50,
            "gender": "男性",
            "prefecture": "福岡県",
            "favorite_sake": ["ウイスキー", "カクテル"],
            "interests": ["ゴルフ", "美術鑑賞"],
            "character": "特別な日に飲む高級な日本酒が好きです。",
            "pass": "ichiro333" # パスワード
        },
        {
            "id": 7,
            "name": "生酛麗子",
            "age": 38,
            "gender": "女性",
            "prefecture": "秋田県",
            "favorite_sake": ["日本酒"],
            "interests": ["陶芸", "温泉"],
            "character": "伝統的な製法の日本酒に興味があります。",
            "pass": "reiko444" # パスワード
        },
        {
            "id": 8,
            "name": "山廃健太",
            "age": 29,
            "gender": "男性",
            "prefecture": "石川県",
            "favorite_sake": ["焼酎", "ビール"],
            "interests": ["キャンプ", "釣り"],
            "character": "個性的な味わいの日本酒が好きです。",
            "pass": "kenta555" # パスワード
        },
        {
            "id": 9,
            "name": "にごり愛",
            "age": 22,
            "gender": "女性",
            "prefecture": "北海道",
            "favorite_sake": ["梅酒", "カクテル"],
            "interests": ["スイーツ", "アニメ"],
            "character": "甘くて飲みやすい日本酒が好きです。",
            "pass": "ai666" # パスワード
        },
        {
            "id": 10,
            "name": "古酒豊",
            "age": 55,
            "gender": "男性",
            "prefecture": "兵庫県",
            "favorite_sake": ["ウイスキー", "ワイン (赤)"],
            "interests": ["ワイン", "旅行", "読書"],
            "character": "熟成された奥深い味わいの日本酒が好きです。",
            "pass": "yutaka777" # パスワード
        }
    ]
}

## イベントデータ

In [ ]:
EVENTS = {
    "events": [
        {
            "id": 1,
            "title": "長岡酒まつり2025",
            "date": "2025-04-01",
            "location": "長岡駅前広場",
            "description": "長岡の地酒が一堂に会するイベント"
        },
        {
            "id": 2,
            "title": "信濃川河川敷でキャンプをしながら夜景鑑賞＆星空観察ツアー",
            "date": "2025-05-10",
            "location": "信濃川河川敷",
            "description": "夜景と星空を楽しむツアー。ロマンチックな雰囲気で交流しやすい環境。"
        },
        {
            "id": 3,
            "title": "吉乃川ミュージアム＆摂田屋を歩こうツアー",
            "date": "2025-11-05",
            "location": "摂田屋エリア",
            "description": "歴史ある醸造の町・摂田屋を巡り、発酵文化を体験するツアー。"
        },
        {
            "id": 4,
            "title": "アルビレックスバスケットボールイベント",
            "date": "2025-06-15",
            "location": "長岡アリーナ",
            "description": "アルビバスケットボールを応援しながら交流するスポーツイベント。"
        },
        {
            "id": 5,
            "title": "フードトラック＆屋台フェスツアー",
            "date": "2025-07-05",
            "location": "長岡市中心部",
            "description": "屋台やフードトラックで様々なグルメを楽しむ食のイベント。"
        },
        {
            "id": 6,
            "title": "長岡の歴史ミステリーツアー",
            "date": "2025-08-12",
            "location": "長岡市内",
            "description": "地元の伝説や都市伝説を巡るミステリーウォーク。"
        },
        {
            "id": 7,
            "title": "写真散歩＆フォトコンテストツアー",
            "date": "2025-09-20",
            "location": "長岡駅周辺",
            "description": "カメラを持って風景を撮影し、後日フォトコンテストを開催。"
        },
        {
            "id": 8,
            "title": "アイドルと会い推しへの愛を長岡の日本酒・ビール、美味しいお店で語ろうツアー",
            "date": "2025-10-05",
            "location": "長岡市内の飲食店",
            "description": "アイドルイベントと飲食ツアーを融合させたファン交流企画。"
        },
        {
            "id": 9,
            "title": "アルビレックスバスケを一緒に観戦しようツアー",
            "date": "2025-11-10",
            "location": "長岡アリーナ",
            "description": "スポーツファンが集まり、試合観戦と懇親会を楽しむイベント。"
        },
        {
            "id": 10,
            "title": "信濃川でカヌー体験をしよう",
            "date": "2025-06-22",
            "location": "信濃川",
            "description": "自然の中でカヌーを体験しながら交流を深めるアウトドアイベント。"
        },
        {
            "id": 12,
            "title": "日本酒作り体験ツアー（田植え～稲刈り、酒造り１年プログラム）",
            "date": "2025-04-20",
            "location": "長岡市内の酒蔵",
            "description": "日本酒造りを１年かけて体験する長期プログラム。"
        },
        {
            "id": 12,
            "title": "桜と柿川のロマンチック川下りツアー",
            "date": "2025-04-15",
            "location": "柿川",
            "description": "春の桜を眺めながら川下りを楽しむリラックスイベント。"
        },
        {
            "id": 13,
            "title": "春の鋸山登山＆山頂で乾杯ツアー",
            "date": "2025-05-03",
            "location": "鋸山",
            "description": "登山の達成感と山頂での乾杯を楽しむアクティビティ。"
        },
        {
            "id": 14,
            "title": "戊辰戦争勃発地・長岡市の歴史を知ろうツアー",
            "date": "2025-09-10",
            "location": "長岡市内の史跡",
            "description": "長岡の歴史を学びながら、歴史的な視点を深めるツアー。"
        },
        {
            "id": 15,
            "title": "山本五十六を知ろうツアー",
            "date": "2025-10-18",
            "location": "山本五十六記念館",
            "description": "山本五十六の生涯とリーダーシップを学ぶ歴史ツアー。"
        }
    ]
}

## その他データ

In [ ]:
# データ


SAKE_SPOTS = {
    "spots": [
        {
            "id": 1,
            "name": "吉乃川株式会社",
            "description": "1548年創業の老舗酒蔵。酒蔵見学も可能です。",
            "location": "長岡市摂田屋",
            "lat": 37.4367,
            "lng": 138.8395,
            "recommended_sake": ["極上吉乃川", "厳選辛口"],
            "speciality": "麹の温度管理にこだわった醸造"
        }
    ]
}



In [ ]:
prefecture_list = [
    "北海道", "青森県", "岩手県", "宮城県", "秋田県", "山形県", "福島県",
    "茨城県", "栃木県", "群馬県", "埼玉県", "千葉県", "東京都", "神奈川県",
    "新潟県", "富山県", "石川県", "福井県", "山梨県", "長野県", "岐阜県",
    "静岡県", "愛知県", "三重県", "滋賀県", "京都府", "大阪府", "兵庫県",
    "奈良県", "和歌山県", "鳥取県", "島根県", "岡山県", "広島県", "山口県",
    "徳島県", "香川県", "愛媛県", "高知県", "福岡県", "佐賀県", "長崎県",
    "熊本県", "大分県", "宮崎県", "鹿児島県", "沖縄県"
]

In [ ]:
alcohol_list = [
    "ビール",
    "ワイン (赤)",
    "ワイン (白)",
    "日本酒",
    "焼酎",
    "ウイスキー",
    "梅酒",
    "カクテル",
    "ノンアルコールビール",
    "ノンアルコールカクテル"
]

In [ ]:
interests_list= [
    "食べ歩き",
    "酒蔵見学",
    "利き酒",
    "酒蔵巡り",
    "旅行",
    "写真",
    "料理",
    "歴史",
    "音楽",
    "映画",
    "ゴルフ",
    "美術鑑賞",
    "陶芸",
    "温泉",
    "キャンプ",
    "釣り",
    "スイーツ",
    "アニメ",
    "ワイン",
    "カフェ巡り",
    "美容",
    "スポーツ",
    "読書",
    "ゲーム"
]

In [ ]:
restaurants_list = [
    {
        "店舗名": "和創作 灯（あかり） 長岡駅前店",
        "おすすめ郷土料理": "栃尾の油揚げ 納豆添え",
        "説明": "新潟県栃尾地域の名物である大きな油揚げに、納豆を添えて提供する一品。外はカリッと、中はふんわりとした食感が特徴で、吉乃川の日本酒との相性も抜群。"
    },
    {
        "店舗名": "ダイニング いまつ",
        "おすすめ郷土料理": "村上牛すき焼き",
        "説明": "新潟県村上市で育てられた高品質な和牛「村上牛」を使用したすき焼き。柔らかく風味豊かな肉質が特徴で、吉乃川の日本酒とともに贅沢な味わいを楽しめる。"
    },
    {
        "店舗名": "たかつ",
        "おすすめ郷土料理": "へぎそば",
        "説明": "新潟県の伝統的な蕎麦で、布海苔（ふのり）という海藻をつなぎに使ったコシのある麺が特徴。木製の器「へぎ」に美しく盛り付けられ、喉ごしの良さが際立つ。"
    },
    {
        "店舗名": "たこの壺",
        "おすすめ郷土料理": "のっぺい汁",
        "説明": "新潟の郷土料理で、里芋、人参、椎茸などの野菜や魚介類を醤油ベースの出汁で煮込んだとろみのある汁物。家庭的な味わいが魅力で、吉乃川の日本酒とともに心温まるひとときを過ごせる。"
    },
    {
        "店舗名": "ちゃーしゅうや武蔵 長岡駅店",
        "おすすめ郷土料理": "長岡生姜醤油ラーメン",
        "説明": "長岡発祥のラーメンで、生姜の風味が効いた醤油スープが特徴。寒い季節に体を温める一杯として親しまれており、吉乃川の日本酒とともに地元の味を堪能できる。"
    },
    {
        "店舗名": "長岡小嶋屋 CoCoLo長岡店",
        "おすすめ郷土料理": "へぎそば",
        "説明": "布海苔をつなぎに使ったコシのある蕎麦で、独特の喉ごしが魅力。吉乃川の日本酒とともに、伝統的な新潟の味を楽しめる。"
    },
    {
        "店舗名": "あかごま",
        "おすすめ郷土料理": "鮭の焼き漬け",
        "説明": "新潟県村上地域の郷土料理で、鮭を焼いてから特製のタレに漬け込んだ一品。濃厚な味わいが特徴で、吉乃川の日本酒との相性も抜群。"
    },
    {
        "店舗名": "和創作 灯-AKARI-",
        "おすすめ郷土料理": "のどぐろ塩焼き",
        "説明": "高級魚として知られるのどぐろをシンプルに塩焼きにした料理。脂がのった白身の旨味を堪能でき、吉乃川の日本酒とともに贅沢な味わいを楽しめる。"
    },
    {
        "店舗名": "アクアーレ長岡",
        "おすすめ郷土料理": "栃尾のジャンボ油揚げ",
        "説明": "栃尾地域特産の大きな油揚げをカリッと焼き上げた一品。外はパリッと、中はふんわりとした食感が魅力で、吉乃川の日本酒とともに味わうと一層美味しさが引き立つ。"
    },
    {
        "店舗名": "あるふぁ・がーでん",
        "おすすめ郷土料理": "笹団子",
        "説明": "新潟の伝統的な和菓子で、笹の葉で包んだ餡入りの餅。ほのかな笹の香りと甘さが特徴で、食後のデザートとして吉乃川の日本酒とともに楽しめる。"
    }
]

json_data = json.dumps(restaurants_list, ensure_ascii=False, indent=4)


In [ ]:
nagaoka_text = """
長岡市について答える
信濃川の土手でグランピングが出来る
空き家を活用した民泊施設がたくさんある
「原信×日本酒」—地元スーパーのお惣菜で気軽に地酒ペアリング
| お惣菜 | ペアリング地酒 | 理由 |
| --- | --- | --- |
| **栃尾の油揚げ（ねぎ味噌）** | 朝日山（朝日酒造） | 旨味とコクが燗酒にぴったり |
| **へぎそば天ぷらセット** | 吉乃川 吟醸 | そばの風味と天ぷらの油をキレの良い吟醸酒が引き立てる |
| **鮭の味噌焼き** | 越乃景虎 超辛口 | 味噌の甘さと辛口酒の爽快感がマッチ |
| **焼き鳥（塩）** | 緑川 純米 | 鶏の旨味を純米酒のコクが引き立てる |
| **新潟名物のっぺ汁** | 越後鶴亀 特別純米 | 根菜の甘みと米の旨味が合う |
原信の店舗で**「日本酒ペアリングセット」**を販売し、旅行者がホテルや新幹線で楽しめるようにする
- 「**原信 × 長岡酒蔵コラボ弁当**」を開発し、長岡駅や道の駅で販売
- **「地酒×原信おつまみセット」**（小分けお惣菜＋ミニボトル）を駅ナカや空港で展開
長岡の地元スーパー（原信・ナカムラ・ウオロクなど）で地酒×お惣菜ペアリング企画**
- 観光客が「**地元民のリアルな食卓**」を体験できるプランを提供
- *「スーパーで買い物して地酒ペアリング体験」**ツアーを企画し、地元の食文化を深く楽しめる"""


In [ ]:
# マッチング履歴を保持するグローバル変数
MATCHING_HISTORY = {
    "matches": []  # {user1_id, user2_id, timestamp, reason} の形式で保存
}

## 関数

In [ ]:
# 関数
def ai(prompt,model="gpt-4o"
      ):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"

def ai_getList(prompt,
                    model="gpt-4o"
      ):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            response_format={"type": "json_object"},
            messages=[
              {"role": "system",
              "content": "常にJSON形式で応答してください",
              "role": "user", "content": f"{prompt}\nJSON形式で答えろ"}
            ]
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"


def user_registration(name, age, gender, prefecture, favorite_sake, character, interests, passward, state):
    """ユーザー登録機能"""
    user = {
        "id": len(USERS_DB["users"]) + 1,
        "name": name,
        "age": age,
        "gender": gender,
        "prefecture": prefecture,
        "favorite_sake": favorite_sake,
        "character": character,
        "interests": interests,
        "pass": passward
    }
    USERS_DB["users"].append(user)
    new_state = {'username': name, 'login': True}
    return f"ユーザー登録が完了しました！\nようこそ、{name}さん", new_state

def login(username, password, state):
    """ログイン処理"""
    for user in USERS_DB["users"]:
        if user["name"] == username and user["pass"] == password:
            new_state = {'username': username, 'login': True}
            return f"ログイン成功！ ようこそ、{username}さん！", new_state
    return "ログイン失敗。ユーザー名またはパスワードが間違っています。", state

def match_sake_spots(user_preferences):
    """日本酒スポットマッチング機能"""
    matches = []
    for spot in SAKE_SPOTS["spots"]:
        score = 0
        # 好みの日本酒タイプとの一致度をスコア化
        for sake in spot["recommended_sake"]:
            if sake in user_preferences:
                score += 1
        if score > 0:
            matches.append({
                "name": spot["name"],
                "description": spot["description"],
                "match_score": score
            })

    # スコアでソート
    matches.sort(key=lambda x: x["match_score"], reverse=True)

    result = "おすすめの日本酒スポット:\n\n"
    for match in matches:
        result += f"🍶 {match['name']}\n{match['description']}\n\n"

    return result

def get_events():
    """イベント情報取得機能"""
    events = sorted(EVENTS["events"], key=lambda x: x["date"])
    result = "upcoming events:\n\n"
    for event in events:
        result += f"📅 {event['title']}\n"
        result += f"日時: {event['date']}\n"
        result += f"場所: {event['location']}\n"
        result += f"詳細: {event['description']}\n\n"
    return result

def sake_pairing_advisor(dish):
    """日本酒ペアリングアドバイザー機能"""
    # ここでは簡単な実装例を示します
    # 実際のアプリではOpenAI APIなどを使用してより高度な提案が可能です
    pairings = {
        "刺身": "純米大吟醸 - 繊細な味わいと香りが魚の旨味を引き立てます",
        "天ぷら": "純米酒 - さっぱりとした味わいが油っぽさを切ります",
        "焼き鳥": "吟醸酒 - 程よい香りと旨味が焼き鳥の風味を引き立てます"
    }
    return pairings.get(dish, "申し訳ありません。その料理に合うお酒が見つかりませんでした。")

def show_sake_map():
    """日本酒観光MAP表示機能"""
    # 実際のアプリではインタラクティブな地図を表示
    map_info = "長岡の日本酒スポット:\n\n"
    for spot in SAKE_SPOTS["spots"]:
        map_info += f"📍 {spot['name']}\n"
        map_info += f"場所: {spot['location']}\n"
        map_info += f"特徴: {spot['speciality']}\n\n"
    return map_info

def match_users(user_id):
    """ユーザー同士のマッチング機能"""
    user = None
    for u in USERS_DB["users"]:
        if u["id"] == user_id:
            user = u
            break

    if not user:
        return "ユーザーが見つかりませんでした。"

    matches = []
    for other_user in USERS_DB["users"]:
        if other_user["id"] != user["id"]:  # 自分自身は除く
            # 簡単なマッチングロジック: 好みの日本酒と都道府県が一致するかどうか
            common_sake = set(user["favorite_sake"]) & set(other_user["favorite_sake"])
            if common_sake and user["prefecture"] == other_user["prefecture"]:
                matches.append(other_user["name"])

    if matches:
        return f"{user['name']}さんへのおすすめユーザー: {', '.join(matches)}"
    else:
        return "おすすめのユーザーは見つかりませんでした。"

def update_visibility(state):
    is_logged_in = state is not None and state.get('login', False)
    return {
        matching_tab: gr.update(visible=is_logged_in),
        event_tab: gr.update(visible=is_logged_in),
        advice_tab: gr.update(visible=is_logged_in),
        map_tab: gr.update(visible=is_logged_in)
    }



def matching(state):
    """GPT-4を使用したユーザーマッチング機能"""
    try:
        if not state or not state.get('username'):
            return "ログインが必要です。"

        # 現在のユーザー情報を取得
        current_user = None
        for user in USERS_DB["users"]:
            if user["name"] == state['username']:
                current_user = user
                break

        if not current_user:
            return "ユーザー情報の取得に失敗しました。"

        # 他のユーザー情報を取得（現在のユーザーを除く）
        other_users = [u for u in USERS_DB["users"] if u["name"] != current_user["name"]]

        prompt = f"""あなたは日本酒好きのマッチングアドバイザーです。以下のユーザー情報を分析し、最適なマッチングを提案してください。
コードブロックやマークダウン記法は使用せず、純粋なJSONオブジェクトのみを返してください。

現在のユーザー:
名前：{current_user["name"]}
年齢：{current_user["age"]}歳
性別：{current_user["gender"]}
好きなお酒：{", ".join(current_user["favorite_sake"])}
興味：{", ".join(current_user["interests"])}

候補となる2名のユーザー:
{json.dumps([{
    "name": u["name"],
    "age": u["age"],
    "gender": u["gender"],
    "favorite_sake": u["favorite_sake"],
    "interests": u["interests"]
} for u in other_users[:2]], ensure_ascii=False, indent=2)}

以下の形式で回答してください:
{{"matched_user": "マッチするユーザー名", "reason": "マッチング理由を1文で", "common_interests": ["共通の興味1", "共通の興味2"], "recommendation": "長岡でのおすすめ活動を1文で"}}"""

        # GPT-4に問い合わせ
        response = ai(prompt)

        # レスポンスの前処理
        # コードブロックマーカーの削除
        response = response.replace("```json", "").replace("```", "").strip()
        print(response)

        try:
            result = json.loads(response)

            # 結果を整形
            output = f"""🎯 マッチング結果

👥 マッチしたユーザー：{result['matched_user']}

📝 理由：
{result['reason']}

🤝 共通点：{', '.join(result['common_interests'])}

💡 おすすめ:
{result['recommendation']}

----------------------------------------"""

            # マッチング履歴に追加
            matched_user = next((u for u in other_users if u["name"] == result["matched_user"]), None)
            if matched_user:
                MATCHING_HISTORY["matches"].append({
                    "user1_id": current_user["id"],
                    "user2_id": matched_user["id"],
                    "timestamp": datetime.now().isoformat(),
                    "reason": result["reason"]
                })

            return output

        except json.JSONDecodeError as e:
            print(f"JSON Parse Error: {e}")
            print(f"Cleaned Response: {response}")  # デバッグ用
            return f"マッチング処理に失敗しました。しばらく待ってから再度お試しください。"

    except Exception as e:
        print(f"General Error: {str(e)}")
        return f"エラーが発生しました: {str(e)}"

def get_logged_in_user_info(login_state):
    """ログインしているユーザーの情報を取得する関数"""
    username = login_state.get('username')  # state から username を取得
    if username:
        for user in USERS_DB["users"]:
            if user["name"] == username:
                return user  # ユーザー情報を含む辞書を返す
    return None  # ログインしていない場合はNoneを返す



def summarize_user_personality(user_data):
    """ユーザーデータに基づいて人物像を要約する関数"""
    prompt = f"""
    以下のユーザー情報を元に、おススメの長岡市の日本酒のおススメベスト３を紹介して。

    名前：{user_data["name"]}
    年齢：{user_data["age"]}歳
    性別：{user_data["gender"]}
    好きなお酒：{", ".join(user_data["favorite_sake"])}
    興味：{", ".join(user_data["interests"])}
    自己紹介：{user_data["character"]}

    最後にベスト１のお酒が飲める長岡市のおススメのお店を紹介して。
    参考にリストを付けるけど、リスト以外のお店でも良い
    """+json_data
    summary = ai(prompt)  # AIにプロンプトを送信し、要約結果を取得
    return summary

def show_user_summary(login_state):
    """ログインユーザーの要約を表示する関数"""
    user_data = get_logged_in_user_info(login_state)

    if user_data:

        summary = summarize_user_personality(user_data)

        return summary  # 要約結果を返す
    else:
        return "ユーザーはログインしていません"

# Google マップの表示用HTMLを動的に生成する関数
def update_map(selected_categories=["キャンプ", "宿泊"]):
    base_query = "新潟県 長岡市"
    if selected_categories:
        query = base_query + " " + " ".join(selected_categories)
    else:
        query = base_query
    query_encoded = urllib.parse.quote_plus(query)
    url = f"https://www.google.com/maps/embed/v1/search?key={a}&q={query_encoded}"
    html_code = f"""
    <iframe
      src="{url}"
      width="100%"
      height="450"
      style="border:0;"
      allowfullscreen=""
      loading="lazy">
    </iframe>
    """
    return html_code

# 出力を生成する関数（input1 の質問に対して外部の ai() 関数を呼び出す）
def display_outputs(question):
    # 長岡市の情報を元に、質問に対して ai() で返答を生成
    response = ai(nagaoka_text + "を元に次の質問に答えて " + question)
    return response

## Gradioインターフェース

In [ ]:
with gr.Blocks(title="長岡日本酒マッチングアプリ") as app:
    gr.Markdown("# 長岡日本酒マッチングアプリ")

    # ログイン状態を保持するStateコンポーネント
    login_state = gr.State(None)

    login_message = gr.Markdown("ログインしてください")

    with gr.Tab("ログイン") as login_tab:
        with gr.Column():
            username_input = gr.Textbox(label="ユーザー名")
            password_input = gr.Textbox(label="パスワード", type="password")
            login_button = gr.Button("ログイン")
            login_output = gr.Textbox(label="ログイン結果")

            login_button.click(
                login,
                inputs=[username_input, password_input, login_state],
                outputs=[login_output, login_state]
            )

    with gr.Tab("ユーザー登録") as register_tab:
        with gr.Column():
            name_input = gr.Textbox(label="名前")
            age_input = gr.Number(label="年齢")
            gender_input = gr.Radio(["男性", "女性", "その他"], label="性別")
            prefecture_input = gr.Dropdown(choices=prefecture_list, label="都道府県")
            sake_input = gr.CheckboxGroup(choices=alcohol_list, label="好きなお酒")
            interests_input = gr.CheckboxGroup(choices=interests_list, label="興味のあること")
            character_input = gr.Textbox(label="自己紹介", lines=3)
            passward_input = gr.Textbox(label="パスワード", type="password")
            register_btn = gr.Button("登録")
            register_output = gr.Textbox(label="登録結果")

            register_btn.click(
                user_registration,
                inputs=[name_input, age_input, gender_input, prefecture_input, sake_input, character_input, interests_input, passward_input, login_state],
                outputs=[register_output, login_state]
            )


    # タブを定義 (初期状態では非表示)
    with gr.Tab("マッチング", visible=False) as matching_tab:
        with gr.Column():
            # GPT-4によるマッチング
            gpt_match_btn = gr.Button("AIによるマッチング")
            gpt_match_output = gr.Textbox(label="マッチング結果")




        # イベントハンドラーの追加
        gpt_match_btn.click(
            matching,
            inputs=[login_state],
            outputs=gpt_match_output
        )

    with gr.Tab("イベント情報", visible=False) as event_tab:
        with gr.Column():
            events_btn = gr.Button("イベント情報を取得")
            events_output = gr.Textbox(label="イベント情報")

        events_btn.click(
            get_events,
            inputs=[],
            outputs=events_output
        )

    with gr.Tab("日本酒アドバイザー", visible=False) as advice_tab:
        with gr.Column():
            advice_btn = gr.Button("あなたにピッタリの日本酒はこちら")
            advice_output = gr.Textbox(label="提案結果")


        # advice_btn のクリック時の処理を修正
        advice_btn.click(
            show_user_summary,
            inputs=[login_state],
            outputs=[advice_output]
        )

    with gr.Tab("観光MAP", visible=False) as map_tab:
      with gr.Column():
            # カテゴリ選択用チェックボックス
            categories = gr.CheckboxGroup(
                choices=["キャンプ", "宿泊", "酒蔵", "酒店", "酒ミュージアム"],
                label="表示したい施設を選択",
                value=["キャンプ", "宿泊"]
            )
            # マップ表示用HTMLコンポーネント
            map_html = gr.HTML(update_map(categories.value))
            categories.change(fn=update_map, inputs=categories, outputs=map_html)

            # 質問入力と回答出力
            input1 = gr.Textbox(label="日本酒の楽しみ方を聞いてください", placeholder="例：地元ならではの飲み方は？")
            submit = gr.Button("Submit")
            output1 = gr.Textbox(label="こんな日本酒の楽しみ方があります")

            # ボタン押下で ai 関数を利用して回答を生成

            submit.click(display_outputs, inputs=input1, outputs=output1)

    def update_visibility(state):
        is_logged_in = state is not None and state.get('login', False)
        return {
            matching_tab: gr.update(visible=is_logged_in),
            event_tab: gr.update(visible=is_logged_in),
            advice_tab: gr.update(visible=is_logged_in),
            map_tab: gr.update(visible=is_logged_in),
            login_message: gr.update(value=f"ようこそ、{state['username']}さん！" if is_logged_in else "ログインしてください")
        }

    # ログイン状態の変化に応じてタブの表示を更新
    login_state.change(
        update_visibility,
        inputs=[login_state],
        outputs=[matching_tab, event_tab, advice_tab, map_tab, login_message]
    )

## Main

In [ ]:
# アプリの起動
if __name__ == "__main__":
    app.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://380396c10b6d659e60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


{"matched_user": "日本酒花子", "reason": "酒蔵見学という共通の興味があり、年齢の差が少ないため", "common_interests": ["酒蔵見学"], "recommendation": "長岡でのおすすめ活動は、有名な酒蔵を巡りつつ温泉に立ち寄ることをお勧めします。"}
{"matched_user": "酒好き太郎", "reason": "共通の興味である酒蔵見学を持ち、カクテル好きのisodaにとって異なる年齢層の視点が魅力的です。", "common_interests": ["酒蔵見学"], "recommendation": "長岡では、酒蔵見学と音楽イベントを組み合わせたツアーを楽しむことをおすすめします。"}


# テスト

In [ ]:
USERS_DB

In [ ]:
MATCHING_HISTORY

In [ ]:
# prompt: MATCHING_HISTORYからuser1_idとマッチしたユーザー名を取得する

def get_matched_usernames(user1_id):
    """MATCHING_HISTORYからuser1_idとマッチしたユーザー名を取得する関数"""
    matched_usernames = []
    for match in MATCHING_HISTORY["matches"]:
        if match["user1_id"] == user1_id:
            # user2_idからユーザー名を取得する
            for user in USERS_DB["users"]:
                if user["id"] == match["user2_id"]:
                    matched_usernames.append(user["name"])
                    break  # ユーザー名が見つかったらループを抜ける
    return matched_usernames

A=get_matched_usernames(11)
print(type(A))